In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
#sample documents
documents = [
    "Anime is great",
    "Where is hell have you been",
    "Anime is the greatest media ever created",
    "Movie is a media used for generation"
]



In [3]:
query = "Best media ever"

In [4]:
import re

def preprocess_text(text):
  text = text.lower()
  text = re.sub(r'[^\w\s]', '', text)
  return text



In [5]:
preprocessed_doc = [preprocess_text(doc) for doc in documents]

In [6]:
preprocessed_doc

['anime is great',
 'where is hell have you been',
 'anime is the greatest media ever created',
 'movie is a media used for generation']

In [7]:
preprocessed_query = preprocess_text(query)

In [8]:
preprocessed_query

'best media ever'

In [9]:
vector = TfidfVectorizer()

In [10]:
X = vector.fit_transform(preprocessed_doc)

In [11]:
X.toarray()

array([[0.5728925 , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.72664149, 0.        , 0.        , 0.        ,
        0.37919167, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.43551105, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.43551105, 0.43551105,
        0.22726773, 0.        , 0.        , 0.        , 0.        ,
        0.43551105, 0.43551105],
       [0.33570696, 0.        , 0.42580171, 0.42580171, 0.        ,
        0.        , 0.        , 0.42580171, 0.        , 0.        ,
        0.222201  , 0.33570696, 0.        , 0.42580171, 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.45203489,
        0.45203489, 0.        , 0.        , 0.        , 0.        ,
        0.23589056, 0.3563895 , 0.45203489, 0.        , 0.45203489,
        0.        , 0.        ]])

In [12]:
query_embedding = vector.transform([preprocessed_query])

In [13]:
query_embedding.toarray()

array([[0.        , 0.        , 0.        , 0.78528828, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.6191303 , 0.        , 0.        , 0.        ,
        0.        , 0.        ]])

using tfidf instead of hugging face transformer. so that the vector representation is of sparse vector instead of dense vector


In [14]:
similarities = cosine_similarity(X, query_embedding)

In [15]:
similarities

array([[0.        ],
       [0.        ],
       [0.54222344],
       [0.22065154]])

In [16]:
np.argsort(similarities, axis = 0)

array([[0],
       [1],
       [3],
       [2]])

In [17]:
ranked_indices = np.argsort(similarities, axis = 0)[::-1].flatten()

In [18]:
ranked_indices

array([2, 3, 1, 0])

In [19]:
ranked_documents = [documents[i] for i in ranked_indices]

In [20]:
ranked_documents

['Anime is the greatest media ever created',
 'Movie is a media used for generation',
 'Where is hell have you been',
 'Anime is great']

In [21]:
for i, doc in enumerate(ranked_documents):
  print(f"Rank {i} : {doc}")

Rank 0 : Anime is the greatest media ever created
Rank 1 : Movie is a media used for generation
Rank 2 : Where is hell have you been
Rank 3 : Anime is great


In [22]:
docs_path = "/content/Deep Learning Techniques for Time Series Forecasting_ A Comprehensive Guide _ by Huntress Elle _ Medium.pdf"

In [23]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.5 MB/s eta 0:00:00


In [24]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.21
    Uninstalling langchain-core-0.3.21:
      Successfully uninstalled langchain-core-0.3.21
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.9
    Uninstalling langchain-0.3.9:
      Successfully uninstalled langchain-0.3.9


In [26]:
from langchain_community.document_loaders import PyPDFLoader

In [27]:
loader = PyPDFLoader(docs_path)

In [29]:
docs = loader.load()

In [32]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [33]:
splitter = RecursiveCharacterTextSplitter(chunk_size = 200, chunk_overlap = 30)

In [35]:
chunks = splitter.split_documents(docs)

In [36]:
from langchain.embeddings import HuggingFaceEmbeddings

In [39]:
embeddings = HuggingFaceEmbeddings( model_name = "BAAI/bge-base-en-v1.5")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [40]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 2.7 MB/s eta 0:00:00


In [41]:
from langchain.vectorstores import Chroma

In [42]:
vectorstore = Chroma.from_documents(chunks, embeddings)

In [43]:
vectorstore_retriever = vectorstore.as_retriever(search_kwargs = {"k":3})

In [45]:
!pip install rank_bm25

In [46]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

In [47]:
keyword_retriver = BM25Retriever.from_documents(chunks)

based on the similarity search how many sentences to fetch

In [48]:
keyword_retriver.k = 3

In [50]:
retriver = EnsembleRetriever(retrievers = [vectorstore_retriever, keyword_retriver], weights = [0.3,0.7])

In [51]:
model_name = "HuggingFaceH4/zephyr-7b-beta"

In [52]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 7.2 MB/s eta 0:00:00


In [53]:
!pip install accelerate